In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
df = pd.read_csv(r"C:\Users\himan\Desktop\7th GAN\updataDataSets.csv", 
                 index_col='From Date', 
                 parse_dates=['From Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33067 entries, 2019-02-18 00:00:00 to 2022-12-31 00:00:00
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SL NO        33067 non-null  int64  
 1   PM2.5        31251 non-null  float64
 2   PM10         31790 non-null  float64
 3   NO           31396 non-null  float64
 4   NO2          32393 non-null  float64
 5   Nox          32616 non-null  float64
 6   NH3          31705 non-null  float64
 7   SO2          32592 non-null  float64
 8   CO           32616 non-null  float64
 9   Ozone        32473 non-null  float64
 10  Benzene      30961 non-null  float64
 11  Eth-Benzene  16450 non-null  float64
 12  MP-Xylene    22659 non-null  float64
 13  WS           31483 non-null  float64
 14  WD           31528 non-null  float64
 15  SR           20184 non-null  float64
 16  BP           21876 non-null  float64
 17  AT           31462 non-null  float64
 18  RF         

C:\Users\himan\AppData\Local\Temp\ipykernel_14072\4218735721.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(r"C:\Users\himan\Desktop\7th GAN\updataDataSets.csv",


In [3]:
print("Original Dataset:")
print(df.head())

Original Dataset:
                     SL NO  PM2.5   PM10    NO   NO2    Nox    NH3    SO2  \
From Date                                                                   
2019-02-18 00:00:00      1  58.04  81.06  1.81  6.40   9.66  27.18  13.77   
2019-02-18 01:00:00      2  60.00  82.39  1.91  6.08   9.70  26.46  13.64   
2019-02-18 02:00:00      3  55.50  73.42  2.22  6.71  10.86  27.06  13.64   
2019-02-18 03:00:00      4  66.56  96.04  2.79  7.00  12.10  26.34  13.72   
2019-02-18 04:00:00      5  67.08  97.21  3.29  9.45  15.53  25.16  13.80   

                       CO  Ozone  Benzene  Eth-Benzene  MP-Xylene    WS  \
From Date                                                                 
2019-02-18 00:00:00  0.49  15.73     0.92          NaN       0.33  1.86   
2019-02-18 01:00:00  0.44  18.36     0.84          NaN       0.29  1.54   
2019-02-18 02:00:00  0.44  34.54     0.77          NaN       0.23  0.66   
2019-02-18 03:00:00  0.45  31.85     0.82          NaN       0.21  

In [4]:
df = df.fillna(0) 

In [5]:
# Define the generator model
def build_generator(latent_dim, n_features):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(n_features, activation='linear'))
    return model

In [6]:
# Define the discriminator model
def build_discriminator(n_features):
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, input_dim=n_features, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [7]:
# Define the Conditional Sig-Wasserstein GAN
def build_cswgan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [8]:
# Define loss functions
def wasserstein_loss(y_true, y_pred):
    return tf.keras.backend.mean(y_true * y_pred)

In [9]:
# Compile the models
latent_dim = 100  # Adjust as needed
generator = build_generator(latent_dim, df.shape[1])
discriminator = build_discriminator(df.shape[1])

In [10]:
# Compile the models
latent_dim = 150
generator = build_generator(latent_dim, df.shape[1])
discriminator = build_discriminator(df.shape[1])

In [11]:
 #Function to generate random noise for the generator
def generate_latent_samples(batch_size, latent_dim):
    return np.random.randn(batch_size, latent_dim)

# Function to get a random batch from the real data
def get_real_samples(data, batch_size):
    idx = np.random.randint(0, data.shape[0], batch_size)
    return data[idx]

# Function to train the Conditional Sig-Wasserstein GAN
def train_cswgan(generator, discriminator, cswgan, real_data, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        for _ in range(len(real_data) // batch_size):
            # Train discriminator
            real_samples = get_real_samples(real_data, batch_size)
            fake_samples = generator.predict(generate_latent_samples(batch_size, latent_dim))
            
            discriminator_loss_real = discriminator.train_on_batch(real_samples, -np.ones((batch_size, 1)))
            discriminator_loss_fake = discriminator.train_on_batch(fake_samples, np.ones((batch_size, 1)))
            discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

            # Train generator (via the combined model)
            latent_samples = generate_latent_samples(batch_size, latent_dim)
            valid_labels = -np.ones((batch_size, 1))
            generator_loss = cswgan.train_on_batch(latent_samples, valid_labels)

            # Print progress
            print(f"Epoch {epoch + 1}, Batch {_ + 1}/{len(real_data) // batch_size}, D Loss: {discriminator_loss}, G Loss: {generator_loss}")


In [12]:
latent_dim = 150
batch_size = 64
epochs = 10

In [13]:
real_data = df.drop(['SL NO'], axis=1).values

In [14]:
# Build and compile the cswgan model
generator = build_generator(latent_dim, real_data.shape[1])
discriminator = build_discriminator(real_data.shape[1])
cswgan = build_cswgan(generator, discriminator)
discriminator.compile(optimizer='adam', loss=wasserstein_loss)
cswgan.compile(optimizer='adam', loss=wasserstein_loss)

# Train the Conditional Sig-Wasserstein GAN
train_cswgan(generator, discriminator, cswgan, real_data, latent_dim, epochs, batch_size)


2/2 [==============================] - 0s 6ms/step

Epoch 1, Batch 1/516, D Loss: 0.23854487016797066, G Loss: -0.47842010855674744
2/2 [==============================] - 0s 4ms/step
Epoch 1, Batch 2/516, D Loss: 0.2573514925315976, G Loss: -0.5314461588859558
2/2 [==============================] - 0s 3ms/step
Epoch 1, Batch 3/516, D Loss: 0.28323200554586947, G Loss: -0.5719108581542969
2/2 [==============================] - 0s 4ms/step
Epoch 1, Batch 4/516, D Loss: 0.30526203545741737, G Loss: -0.6119959354400635
2/2 [==============================] - 0s 4ms/step
Epoch 1, Batch 5/516, D Loss: 0.32440510019659996, G Loss: -0.6606863737106323
2/2 [==============================] - 0s 3ms/step
Epoch 1, Batch 6/516, D Loss: 0.3454045783728361, G Loss: -0.7059348821640015
2/2 [==============================] - 0s 4ms/step
Epoch 1, Batch 7/516, D Loss: 0.37033022404648364, G Loss: -0.7528724074363708
2/2 [==============================] - 0s 4ms/step
Epoch 1, Batch 8/516, D Loss: 0.390188

In [15]:
# Generate predictions using the trained generator
latent_samples = np.random.randn(100, latent_dim)  # Adjust as needed
generated_data = generator.predict(latent_samples)

4/4 [==============================] - 0s 7ms/step


In [16]:
# Display the generated data
print(generated_data)

[[-34.391167  -26.94673   -52.933018  ...  56.80726    12.696836
   -8.417642 ]
 [-36.09364   -27.715927  -55.42204   ...  59.604023   13.376146
   -9.168032 ]
 [-32.552982  -24.422543  -49.79395   ...  53.630466   12.180327
   -8.507862 ]
 ...
 [-36.43808   -27.422878  -56.246296  ...  59.776436   13.415622
   -9.743786 ]
 [-31.162361  -23.862991  -47.83083   ...  51.537567   11.606354
   -7.4002347]
 [-34.705692  -25.037792  -51.693043  ...  55.939354   12.070661
   -8.654562 ]]
